In [5]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import os
import requests
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage
from io import BytesIO

# 현재 시간으로 폴더명 만들기
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday,
                                       now.tm_hour, now.tm_min, now.tm_sec)

# 폴더 구조 만들기
query_txt = '마켓컬리_크롤링_제출용'
f_dir = os.getcwd()  # 현재 작업 디렉토리
base_dir = os.path.join(f_dir, f"{s}-{query_txt}")
img_dir = os.path.join(base_dir, "images")

# 폴더 생성
os.makedirs(img_dir, exist_ok=True)
print(f"폴더 생성 완료: {base_dir}")
print(f"이미지 폴더: {img_dir}")

# 크롬 드라이버 실행
driver = webdriver.Chrome()

# 마켓컬리 사이트 접속
driver.get("https://www.kurly.com/")
time.sleep(3)

# 팝업창 닫기 버튼클릭하는것은 XPATH 로 정확히 찾아주기!
try:
    # 팝업 닫기 버튼 (텍스트로 찾기)
    popup_close = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '닫기')]"))
    )
    popup_close.click()
    time.sleep(2)
except:
    print("팝업이 없거나 이미 닫혔습니다")

# 스크롤 살짝만 내리기 (500픽셀)
driver.execute_script("window.scrollBy(0, 500);")
time.sleep(2)

# 전체보기 버튼 클릭(상품 내용 전체보기) 이런 버튼또한 XPATH 로 정확히 찾아주자! 
try:
    # 전체보기 버튼 태그
    view_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/div[4]/div[1]/main/div[4]/a'))
    )
    view_all_button.click()
    time.sleep(3)
except:
    print("전체보기 버튼을 찾을 수 없습니다")

# 데이터를 저장할 리스트
ranking_list = []        # 랭킹 순번
product_name_list = []   # 상품 이름
product_desc_list = []   # 상품 설명
discount_list = []       # 할인율
price_list = []          # 상품 가격
image_url_list = []      # 이미지 URL
kurly_only_list = []     # Kurly Only 여부

# 3페이지 크롤링(한페이지에 96개가 있으니 랭킹순위 200개를 뽑기위해서 3페이지넘어가야된다.)
for page in range(1, 4):
    print(f"{page}페이지 크롤링 중...")
    time.sleep(3)
    
    # 스크롤을 천천히 내려서 이미지 로드 (새로 추가!)
    print("이미지 로드를 위해 스크롤 중...")
    
    # 페이지 끝까지 스크롤했을 때의 전체 길이(px) 를 알려주는 코드이다.
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 500픽셀씩 스크롤
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(0.5)  # 이미지/데이터 로드 기다림(천천히가자!)
        
        # 현재 스크롤 위치 <-필요한이유: 스크롤 종료할타이밍을 알아야하니까
        new_height = driver.execute_script("return window.pageYOffset + window.innerHeight")
        
        # 끝까지 스크롤했으면 종료
        if new_height >= last_height:
            break
    
    print("스크롤 완료! 이미지 로드 대기 중...")
    time.sleep(3)  # 마지막 이미지 로드 대기
    
    # 맨 위로 스크롤 (다시맨위로올라가서 상품 수집하기!)
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    
    # 클래스(class)를 안 쓰고, 태그(tag)만으로 필요한 정보(img, h4, p, span 등)를 각각 찾아서 꺼내는 방식을사용함.
    products = driver.find_elements(By.TAG_NAME, "article") #<-1개의 상품을 감싸는 큰 컨테이너(article)
    print(f"찾은 상품 수: {len(products)}")
    
    for idx, product in enumerate(products): #<-enumerate반복문에서 자동으로 순번을 붙여주는 함수
        try:
            # 현재 페이지에서의 순위 계산
            ranking = (page - 1) * 96 + (idx + 1)
            
            # 이미지 URL 태그 는 TAG_NAME 으로 찾기
            image = product.find_element(By.TAG_NAME, "img")
            image_url = image.get_attribute("src")
            
            # 상품 이름 태그는 h4 태그로 찾기
            try:
                name = product.find_element(By.TAG_NAME, "h4")
                product_name = name.text
            except:
                product_name = "상품명 없음"
                print(f"상품 {idx+1}: 상품 이름을 찾을 수 없습니다")
            
            # 상품 설명 태그 는 p 태그로 찾기
            try:
                desc = product.find_element(By.TAG_NAME, "p")
                product_desc = desc.text
            except:
                product_desc = ""
            
            # 할인율 태그 는 % 포함된 span 찾기
            try:
                spans = product.find_elements(By.TAG_NAME, "span")
                discount_rate = "할인없음"
                for span in spans:
                    if "%" in span.text:
                        discount_rate = span.text
                        break
            except:
                discount_rate = "할인없음"
            
            # 가격 태그 는 "원" 포함된 span 찾기
            try:
                spans = product.find_elements(By.TAG_NAME, "span")
                product_price = "가격 정보 없음"
                for span in spans:
                    if "원" in span.text:
                        product_price = span.text
                        break
            except:
                product_price = "가격 정보 없음"
            
            # Kurly Only 텍스트로 찾기
            try:
                spans = product.find_elements(By.TAG_NAME, "span")
                is_kurly_only = ""
                for span in spans:
                    if "Kurly Only" in span.text:
                        is_kurly_only = "Kurly Only"
                        break
            except:
                is_kurly_only = ""  #<- 없으면 빈칸으로 두기
            
            # 리스트에 추가
            ranking_list.append(ranking)
            product_name_list.append(product_name)
            product_desc_list.append(product_desc)
            discount_list.append(discount_rate)
            price_list.append(product_price)
            image_url_list.append(image_url)
            kurly_only_list.append(is_kurly_only)
            
        except Exception as e:
            print(f"상품 {idx+1} 크롤링 중 오류: {e}")
            continue
    
    # 200개 이상 수집했으면 중단
    if len(ranking_list) >= 200:
        break
    
    # 다음 페이지로 이동하기위해 if 문으로 페이지버튼 돌리기.
    if page < 3:
        try:
            # 다음 페이지 버튼 태그
            next_button = driver.find_element(By.XPATH, '//*[@id="container"]/div/div/div[2]/a[4]')
            next_button.click()
            time.sleep(3)
        except:
            print("다음 페이지 버튼을 찾을 수 없습니다")
            break

# 우리는 랭킹 200을 뽑을꺼니까, 200개까지만 자르기
ranking_list = ranking_list[:200]
product_name_list = product_name_list[:200]
product_desc_list = product_desc_list[:200]
discount_list = discount_list[:200]
price_list = price_list[:200]
image_url_list = image_url_list[:200]
kurly_only_list = kurly_only_list[:200]

# 데이터프레임 생성
df = pd.DataFrame({
    "순위": ranking_list,
    "이미지URL": image_url_list,
    "상품이름": product_name_list,
    "상품설명": product_desc_list,
    "할인율": discount_list,
    "가격": price_list,
    "Kurly Only": kurly_only_list
})

# 엑셀 파일명 (폴더 안에 저장)
excel_file = os.path.join(base_dir, f"{s}-{query_txt}.xlsx")

# 기본 엑셀 파일 저장 (이미지 없이)
df.to_excel(excel_file, index=False)
print(f"기본 엑셀 파일 저장 완료: {excel_file}")

# 이미지 다운로드 및 엑셀에 삽입
print("\n이미지 다운로드 및 삽입 시작...")
wb = load_workbook(excel_file)
ws = wb.active

# B열 너비 조정 (이미지가 들어갈 공간)
ws.column_dimensions['B'].width = 20 # 사진이 들어 갈 수있게 엑셀의 넓이를 20px 넓힌다 

success_count = 0  # 성공 카운트
skip_count = 0     # 스킵 카운트

for idx, img_url in enumerate(image_url_list, start=2):  # 리스트를 돌면서 순번도 같이 가져옴!
    try:
        # data URI는 건너뛰기 (플레이스홀더 이미지)
        if img_url.startswith('data:image'):
            skip_count += 1
            print(f"순위 {ranking_list[idx-2]}: 플레이스홀더 이미지 건너뜀")
            continue
        
        # HTTP/HTTPS URL만 처리
        if not img_url.startswith('http'):
            skip_count += 1
            continue
        
        # 행 높이 조정
        ws.row_dimensions[idx].height = 100 # 사진이 들어갈 수 있게 행의 높이를 넓힘 
        
        # 이미지 다운로드
        response = requests.get(img_url, timeout=10)
        if response.status_code == 200:
            # 이미지 저장 (로컬 파일로)
            img_filename = f"product_{ranking_list[idx-2]}.jpg"
            img_path = os.path.join(img_dir, img_filename)
            
            with open(img_path, 'wb') as f:
                f.write(response.content)
            
            # 엑셀에 이미지 삽입
            img = XLImage(img_path) 
            # 집어넣을 이미지 크기 조정 (너비 100픽셀로)
            img.width = 100
            img.height = 100
            
            # B열에 이미지 삽입 (B2, B3, B4...)
            ws.add_image(img, f'B{idx}') 
            success_count += 1
            
            if success_count % 20 == 0:  # 20개마다 진행상황 출력
                print(f"이미지 처리 중... {success_count}개 완료")
        else:
            skip_count += 1
            print(f"순위 {ranking_list[idx-2]}: 이미지 다운로드 실패 (상태코드: {response.status_code})")
    
    except Exception as e:    # ←오류가 나도 멈추지않고 실패로간주후 다음으로 넘어가자
        skip_count += 1
        print(f"순위 {ranking_list[idx-2]}: 이미지 처리 오류 - {str(e)[:100]}")
        continue

# 최종 엑셀 저장
wb.save(excel_file)  #지금까지 엑셀에 삽입한 모든 이미지를 실제파일에 덮어쓰기(저장)
print(f"\n✅ 이미지 삽입 완료!")
print(f"   - 성공: {success_count}개")
print(f"   - 건너뜀: {skip_count}개")
print(f"   - 최종 파일: {excel_file}")

print(f"\n크롤링 완료! 총 {len(df)}개 상품 수집")
print(f"파일 저장 위치: {excel_file}")

# ==================== Kurly Only 상품만 필터링 ====================
kurly_only_df = df[df["Kurly Only"] == "Kurly Only"].copy()
kurly_only_file = os.path.join(base_dir, f"{s}-KurlyOnly_상품.xlsx")
kurly_only_df.to_excel(kurly_only_file, index=False)

print(f"\n Kurly Only 상품: {len(kurly_only_df)}개")
print(f"Kurly Only 파일 저장: {kurly_only_file}")

# ==================== Kurly Only 파일에도 이미지 삽입 ====================
print("\nKurly Only 파일에 이미지 삽입 시작...")
wb_kurly = load_workbook(kurly_only_file)
ws_kurly = wb_kurly.active
ws_kurly.column_dimensions['B'].width = 20

success_count_kurly = 0
skip_count_kurly = 0

# Kurly Only 데이터프레임의 순위를 사용해서 이미지 찾기
for idx, row in enumerate(kurly_only_df.itertuples(), start=2):
    try:
        img_url = row.이미지URL
        ranking = row.순위
        
        if img_url.startswith('data:image') or not img_url.startswith('http'):
            skip_count_kurly += 1
            continue
        
        ws_kurly.row_dimensions[idx].height = 100
        
        # 이미 다운로드된 이미지 파일 사용
        img_filename = f"product_{ranking}.jpg"
        img_path = os.path.join(img_dir, img_filename)
        
        # 이미지 파일이 존재하는지 확인
        if os.path.exists(img_path):
            img = XLImage(img_path)
            img.width = 100
            img.height = 100
            ws_kurly.add_image(img, f'B{idx}')
            success_count_kurly += 1
            
            if success_count_kurly % 20 == 0:
                print(f"  이미지 처리 중... {success_count_kurly}개 완료")
        else:
            skip_count_kurly += 1
    
    except Exception as e:
        skip_count_kurly += 1
        continue

wb_kurly.save(kurly_only_file)
print(f"\n Kurly Only 이미지 삽입 완료!")
print(f"   - 성공: {success_count_kurly}개")
print(f"   - 건너뜀: {skip_count_kurly}개")

print(f"\n크롤링 완료! 총 {len(df)}개 상품 수집")
print(f"파일 저장 위치:")
print(f"   - 전체 상품 (이미지 포함): {excel_file}")
print(f"   - Kurly Only (이미지 포함): {kurly_only_file}")

driver.quit()


폴더 생성 완료: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용
이미지 폴더: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용\images
1페이지 크롤링 중...
이미지 로드를 위해 스크롤 중...
스크롤 완료! 이미지 로드 대기 중...
찾은 상품 수: 96
2페이지 크롤링 중...
이미지 로드를 위해 스크롤 중...
스크롤 완료! 이미지 로드 대기 중...
찾은 상품 수: 52
3페이지 크롤링 중...
이미지 로드를 위해 스크롤 중...
스크롤 완료! 이미지 로드 대기 중...
찾은 상품 수: 52
기본 엑셀 파일 저장 완료: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용\2025-10-21-15-44-43-마켓컬리_크롤링_제출용.xlsx

이미지 다운로드 및 삽입 시작...
이미지 처리 중... 20개 완료
이미지 처리 중... 40개 완료
이미지 처리 중... 60개 완료
이미지 처리 중... 80개 완료
이미지 처리 중... 100개 완료
이미지 처리 중... 120개 완료
이미지 처리 중... 140개 완료
이미지 처리 중... 160개 완료
이미지 처리 중... 180개 완료
이미지 처리 중... 200개 완료

✅ 이미지 삽입 완료!
   - 성공: 200개
   - 건너뜀: 0개
   - 최종 파일: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용\2025-10-21-15-44-43-마켓컬리_크롤링_제출용.xlsx

크롤링 완료! 총 200개 상품 수집
파일 저장 위치: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용\2025-10-21-15-44-43-마켓컬리_크롤링_제출용.xlsx

 Kurly Only 상품: 141개
Kurly Only 파일 저장: c:\py_temp\2025-10-21-15-44-43-마켓컬리_크롤링_제출용\2025-10-21-15-44-43-KurlyOnly_상품